# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task

## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [6]:
import pandas as pd
import numpy as np
import openai

df = pd.read_csv('2023_fashion_trends.csv')

df['Trends']

0     2023 Fashion Trend: Red. Glossy red hues took ...
1     2023 Fashion Trend: Cargo Pants. Utilitarian w...
2     2023 Fashion Trend: Sheer Clothing. "Bare it a...
3     2023 Fashion Trend: Denim Reimagined. From dou...
4     2023 Fashion Trend: Shine For The Daytime. The...
                            ...                        
77    If lime green isn't your vibe, rest assured th...
78    "As someone who can clearly (not fondly) remem...
79    "Combine this design shift with the fact that ...
80    Thought party season ended at the stroke of mi...
81    "This season, we saw the revival of the bubble...
Name: Trends, Length: 82, dtype: object

In [41]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
response = openai.embeddings.create(input = df["Trends"], model=EMBEDDING_MODEL_NAME)

In [98]:
for i,data in enumerate(response.data):
    df["embeddings"][i] = data.embedding
    #print(i,response.data[0].embedding)

In [99]:
df["embeddings"]

0     [-0.020776711404323578, -0.021952012553811073,...
1     [-0.001848788931965828, -0.02892092987895012, ...
2     [-0.010463573038578033, -0.019268615171313286,...
3     [-0.015569761395454407, -0.005440722219645977,...
4     [-0.004972496535629034, 0.0018217908218502998,...
                            ...                        
77    [-0.0027773624751716852, -0.01823779009282589,...
78    [-0.014705789275467396, -0.006454960443079472,...
79    [-0.020781872794032097, -0.025097381323575974,...
80    [-0.019823411479592323, -0.022405004128813744,...
81    [-0.030918996781110764, 0.002061266452074051, ...
Name: embeddings, Length: 82, dtype: object

In [56]:
Query = openai.embeddings.create(input = "What are the fashion trends for 2023?", model=EMBEDDING_MODEL_NAME)

In [104]:
Query_embeddings = Query.data[0].embedding

AttributeError: 'list' object has no attribute 'numpy'

In [103]:
from embeddings_utils import distances_from_embeddings

distances = distances_from_embeddings(Query_embeddings, Query_embeddings)


ValueError: Input vector should be 1-D.

## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [37]:
def get_answer(role, question):
   gptrole = {"role": "system", "content": role }
   gptquestion = {"role": "user", "content": question}
   messages = [gptrole, gptquestion]
   print(messages)
   response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages
    )
   return response.choices[0].message.content

In [38]:
answer = get_answer("You are a helpful assistant", "What are the fashion trends for 2023?")

[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What are the fashion trends for 2023?'}]


In [39]:
answer

"Predicting fashion trends can be quite challenging as they are subject to change based on various factors such as influences from social media, celebrities, culture, and global events. However, some common trends that are expected to emerge in 2023 include:\n\n1. Sustainability and Eco-Friendly Fashion: With an increased focus on environmental issues, sustainable and eco-friendly fashion will continue to be a key trend in 2023. This includes the use of organic and recycled materials, as well as ethical production practices.\n\n2. Gender-Neutral Fashion: The rise of gender-neutral clothing and unisex styles is expected to continue in 2023, as the fashion industry becomes more inclusive and diverse.\n\n3. Minimalism and Classic Styles: Timeless pieces and minimalist designs are expected to be popular in 2023, as people seek longevity and versatility in their wardrobe choices.\n\n4. Bold Colors and Prints: Vibrant colors, bold prints, and patterns are expected to make a statement in 2023

In [ ]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string("tiktoken is great!", "cl100k_base")

In [30]:
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "when was denim reimagined in fashion?"},
    ]

In [32]:
print(messages)

[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'when was denim reimagined in fashion?'}]


## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

### Question 2